<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/running_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [ ]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [ ]:
!pip install -q canopy

In [ ]:
import canopy
import logging
import nest_asyncio

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

### Authenticate

In [ ]:
authentication_data = canopy.prompt_for_authentication()
session = canopy.Session(authentication_data)

# Set Up Example

Create the input configs for our study:

In [ ]:
# Load a default car and save it as a user config.
default_car = await canopy.load_default_config(
    session, 'car', 'Canopy F1 Car 2019')

user_car_id = await canopy.create_config(
    session,
    'car',
    'Running Studies Example Car',
    default_car.raw_data)

# Load a default weather and save it as a user config.
default_weather = await canopy.load_default_config(
    session, 'weather', '25 deg, dry')

user_weather_id = await canopy.create_config(
    session,
    'weather',
    'Running Studies Example Weather',
    default_weather.raw_data)

# Load the user weather config.    
user_weather = await canopy.load_config(session, user_weather_id)

# Load a default exploration and reduce its size for this example.
default_exploration = await canopy.load_default_config(
    session, 'exploration', 'Automated Test Monte Carlo')
default_exploration.data.design.numberOfPoints = 3

> **Tip: `data` vs `raw_data`**
>
> We could use either the `default_car.raw_data` property or the `default_car.data` property when creating the user config above. 
> 
> On first access, the `data` property will convert the nested dictionaries (`car['a']['b']`) into a structure which we can manipulate more easily (`car.a.b`). 
> 
> While convenient, this conversion does have some overhead, particularly on large configs such as tracks. As we are not manipulating the data here it is good practice to skip the conversion by using the `raw_data` property instead. 
> 
> The `raw_data` property will return either:
 - The same structure as `data` if the conversion has already been done.
 - The original nested dictionaries if the conversion hasn't already been done.




# Example: Running Studies

The `create_study` function takes a list of configs to use as inputs to the study. Each config can be either:
 - A string config ID.
 - A `ConfigResult` class, as returned by functions such as `load_config`.
 - A `LocalConfig` class, as returned by functions such as `load_default_config`.

The code below demonstrates passing one config of each type:

In [ ]:
study_id = await canopy.create_study(
    session,
    'apexSim',
    'Running Studies Example Study',
    [
        user_car_id, # String
        user_weather, # ConfigResult class
        default_exploration, # LocalConfig class
    ])
    
study_id

INFO:canopy.create_study:Loaded input config car


'39b23f0e8d2c4fd89e4dfc23aaf39ff9'

> **Tip: Study Types and Sim Types**
>
> A study contains one or more jobs, and each job contains one or more simulations.
For example a Dynamic Lap study will contain a Dynamic Lap, a Straight Sim and Apex Sim simulation in each job.
> 
> When dealing with the Canopy API study types are `camelCase` and sim types are `PascalCase`.
>
> Therefore a `dynamicLap` study will contain `DynamicLap`, `StraightSim` and `ApexSim` simulations.
>
> Above we are running an `apexSim` study which will contain `ApexSim` and `StraightSim` simulations.
>
> The Python library will tend to ignore case in the helper functions we've written and interpret the string based on the context, however when dealing with the generated Swagger client code directly you will need to use the correct case so it is good practice to always try and do this.


We can use the `wait_for_study` helper function to wait until the study has finished running, which is particularly useful if we want to access the results in our notebook.

In [ ]:
wait_result = await canopy.wait_for_study(
    session,
    study_id,
    timeout_seconds=300)
    
logging.info('Sim Version: {}'.format(wait_result.document.sim_version))    
logging.info('Succeded Simulations: {}/{}'.format(
    wait_result.succeeded_simulation_count, 
    wait_result.simulation_count))

INFO:root:Sim Version: 1.3371
INFO:root:Succeded Simulations: 3/3


### Additional Study Data
You can also pass in custom properties and notes when creating the study. Our library will merge the custom properties and notes from all the input configs into the final study, just like the Canopy portal does.

In [ ]:
user_weather.document.properties = {
    'location': 'london'
}
user_weather.document.notes = 'London was unusually hot.'

logging.info('Submitting study...')

study_id = await canopy.create_study(
    session,
    'apexSim',
    'Running Studies Example Study',
    [
        user_car_id,
        user_weather,
    ],
    properties={
        'foo': 'bar'
    },
    notes='Some notes about the study.')

logging.info('Waiting for study...')

wait_result = await canopy.wait_for_study(
    session,
    study_id,
    timeout_seconds=300)

logging.info('Done.')

INFO:root:Submitting study...
INFO:canopy.create_study:Loaded input config car
INFO:root:Waiting for study...
INFO:root:Done.


Looking at the study custom properties we see the weather custom properties have been merged in, in the same way as if you had run the study on the Canopy Portal.

In [ ]:
wait_result.document.properties

{'foo': 'bar', 'weather.location': 'london'}

The same is true of the study notes. However the `wait_result` only contains the study metadata, so we first need to load the full study document to see the notes.

Once we do this we see that the weather notes have been merged with the study notes, again using the same convention as if you ran the study though the Canopy Portal.

In [ ]:
study = await canopy.load_study(
    session,
    study_id,
    include_study_full_document=True)

print(study.document.notes)

Some notes about the study.

weather:
London was unusually hot.


Because the Python library has merged the notes and custom properties using this standard convention, if you were now to use the Canopy Portal to stage the weather config from the study page, it will successfully extract the weather notes and custom properties and put them in the staged config.

## Getting a Study ID from the Canopy Portal

If you want to load an existing study by ID, you can find its ID by opening it in the Canopy Portal (the web UI to the platform).

The URL for a studies is of the form `/studies/<tenantId>/<studyId>`. You can therefore simply copy the Study ID from the URL and paste it into your Python code.

Similarly for studies the URL is of the form `/configs/<configType>/<tenantId>/<configId>/edit`.

Failing that, you can load the config by name or other metadata, which we'll do next.


## Loading a study by name / metadata

Sometimes it is more convenient to load a study by name:

In [ ]:
study_by_name = await canopy.find_study(
    session, 
    name='Running Studies Example Study')

print(study_by_name.study_id)

a4aeb9ade1134a3c98a1e7dcb8e4fb77


We can also search by other metadata, such as custom properties:

In [ ]:
study_by_property = await canopy.find_study(
    session,
    custom_properties={ 'weather.location': 'london' })

print(study_by_property.study_id)

a4aeb9ade1134a3c98a1e7dcb8e4fb77


If multiple studies match the search criteria, it will return the study which was most recently run.

The object returned from `find_study` is a very minimal set of study metadata. However you can now take the `study_id` property and pass it into `load_study` or `load_study_job` to get the precise data you require.